Consider a small slice of time and of thickness $z$. My intensity is attnuated as $I= I_0 e^{-\sigma*N*z}$ assuming the temporal 'width' of I is shorter than the excited state lifetime of my chromophore ie no re-excitation. After I have passed through this slice the number of molecules that have switched is: going to be proportional to : the cross section of absoption ($\sigma$), yield ($\phi$) and the number of photons (\#): 
$$\Delta N= N \sigma \phi \# $$
So I can consider the probility of just a single photon getting through ? And then my intesnity over that number of photons. The integral of N will give me the rate. First need to know the cross section of rs-Kiiro. The units of cross section per a molecule at 400 nm go as: 
$$ \sigma_i = \frac{ln(10) \epsilon}{N_A}= \frac{M^{-1}cm^{-1}}{mol^{-1}}=\frac{mol^{-1}dm^{3}cm^{-1}}{mol^{-1}} = 1000*cm^{3}cm^{-1}=1000*cm^{2}$$

$$  \sigma_i= \frac{ln10}{N_A}*1000*(\epsilon_{500 nm} /3 )  = \frac{ln10}{N_A}*1000*(115000/3)*M^{-1}*cm^{-1}= 1.47 e-16 cm^{2}  $$

Need to then account for the yield in some way. The cross section accounts for the probabilty of absoption but want to know given an absorption what is the likelyhood of switching. Quantum yield of flourescence is defined as:
    $$ \phi_{Flou} = \frac{\text{# of photons emitted}}{\text{# of photons absorbed}} = 0.59 \text{ (Skylan-NS)}$$ 

This can be used to give an idea of laser induced heating as the thermal contributions of a CW laser can be calucalted in proportion to: 
    $$ \phi_{Thermal}=1-\phi_{Flou}$$
Need to make some assumptions about the switching yield. If can find the Skylan-NS yield then could calculate from the rates: 
    $$ \phi_{Switch,Kiiro}=\frac{k_{Kiiro}}{k_{Skylan-NS}}*\phi_{Switch,Skylan-NS}$$
 
For now will assume: $\phi_{Switch,Kiiro}=0.3$ which seems simular to reported of dronpa ect.. 

Have the population densities being dependant on the number of photons, $\#$. But have written attenuation as a function of intensity. Need the intesnity as number of photons per area per a second. Or.. 
$$ I = \frac{\# }{A*t} $$

Measurements were done with an energy density: $$\rho=\frac{\# E_i}{A}=\frac{It}{A}$$
$$ \# = \frac{\rho A}{E_i} $$
$$ I_0=\frac{\rho}{E_i t}  $$#
For example: energy density of $\rho = 1 mJ/mm^2 $ in a time of $t=20ms$ with wavelength 405 nm therefore $E_i=\frac{hc}{405 nm} = 4.42e-17 J = 4.42 e-14 mJ $ so the the rate of photons per a unit area and time is 
$$ I_0=\frac{100 mJ / cm^2}{4.42e-14mJ*20e-3 s}=1.13e17 \text{ photons per second per cm}^2$$
$$ I_0 = \frac{\rho*100}{4.42e-14*t} $$
What should starting N be ? Know that samples had an average OD of 0.23 at 505 nm. So can use Beer-Lambert:
$$ A=\epsilon l c$$
But want c*l in cm$^{-2}$ not in mol/dm$^{3}$*cm$^{-1}$ so:
$$ lc=\frac{A}{\epsilon}\frac{N_A}{1000} cm^{-2}=1.20e15 cm^{-2}$$
Actually maybe should be considering the initial intensity as a rate of photons per an area: 
$$ I_0=\frac{1}{A}\frac{d\#}{dt}$$

import numpy as np

def pop(N, z,width):
	if z > width:
		u = 0
	elif z < 0:
		u = 0
	else:
		u = 1
	return N * u


def atten(N, z, I0, sig, width):
	if z < 0:
		v = I0
	elif z > width:
		v = I0 * np.exp(-sig * width)
	else:
		v = I0 * np.exp(-sig * N * z)  # Remember to change line above!!
	return v


def delN(N, phi, sig, I):
	return N * phi * sig * I


def I0_rate(ED):
	return ED * 100 / 4.42e-16  # *10e-3 #Intensity per cm2 per s


def model(ED, phi, width, sig, tstep, tmax, N0):
	dI0dt = I0_rate(ED)
	z = np.linspace(-0.1, 1.1, 100)
	t = np.empty(int(tmax / tstep))
	N = np.empty_like(z)
	I = np.empty_like(z)
	dN = np.empty_like(z)
	Nf = np.empty_like(z)
	Intergral = 0

	# Set up time:
	for i in enumerate(t):
		t[i[0]] = tstep

	for j in enumerate(t):
		I0 = dI0dt * j[1]
		if j[0] == 0:
			for i in enumerate(z):
				N[i[0]] = pop(N0, i[1], width)
				I[i[0]] = atten(pop(N0, i[1], width), i[1], I0, sig, width)
				dN[i[0]] = delN(N[i[0]], phi, sig, I[i[0]])
				Nf[i[0]] = N[i[0]] - dN[i[0]]

		else:
			N = Nf
			for i in enumerate(z):
				I[i[0]] = atten(N[i[0]], i[1], I0, sig, width)
				I[i[0]] = atten(pop(N[i[0]], i[1], width), i[1], I0, sig, width)
				I[i[0]] = atten(pop(N[i[0]], i[1], width), i[1], I0, sig, width)
				dN[i[0]] = delN(N[i[0]], phi, sig, I[i[0]])
				Nf[i[0]] = N[i[0]] - dN[i[0]]
	for i in enumerate(z):
		if i[0] > 0:
			Intergral = Intergral + (z[i[0]] - z[i[0] - 1]) * (
					(Nf[i[0]] + Nf[i[0] - 1]) / 2)  # The trapeze rule - there must be a nicer way to do this..
	print("Percent: ", (Intergral / N0))
	return (Intergral / N0, ED)


In [3]:
import attenuation_modeling_funcs as amf
import multiprocessing as mp
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit


def collect_result(result):
	global results
	results.append(result)


print("Number of processors: ", mp.cpu_count())
width = 1.0
N0 = 1.20e15  # sample per cm-2
sig = 1.47e-16  # cross section per cm-2
phi = 0.3  # quantum yeidld of converstion
tstep = 1e-4  # In Seconds
tmax = 20e-3  # in seconds

ED = np.linspace(1, 100, 100)

pool = mp.Pool(mp.cpu_count())
results_object = [pool.apply_async(amf.model, args=(EDi, phi, width, sig, tstep, tmax, N0)) for EDi in ED]
result_ED = np.asarray([r.get()[1] for r in results_object])
result_percent = np.asarray([r.get()[0] for r in results_object])
pool.close()
pool.join()


def exponenial_func(x, a, b, c):
	return a * np.exp(-b * x) + c

popt, pcov = curve_fit(exponenial_func, result_ED, result_percent, p0=(1, 0.1, 0.1))
fit = exponenial_func(result_ED, *popt)
plt.plot(result_ED, result_percent)
plt.plot(result_ED,fit)
plt.show()
# amf.model(ED, phi, width, sig, tstep, tmax, N0)



Number of processors:  4
Percent:  0.6857594795701215
Percent:  0.5676229842276698
Percent:  0.46892297634493246
Percent:  0.826638835159026
Percent:  0.3867282254948814
Percent:  0.3184713196046595
Percent:  0.2151838751882556
Percent:  0.2619269893936026
Percent:  0.17661290340603758
Percent:  0.1448347161177536
Percent:  0.11868789892709648
Percent:  0.0971991490700483
Percent:  0.07955604457220104
Percent:  0.06508271669334093
Percent:  0.053218477106550734
Percent:  0.043499287754724114
Percent:  0.035541864104449705
Percent:  0.02903015296738625
Percent:  0.023703909051950005
Percent:  0.019349098370489568
Percent:  0.01288188587878721
Percent:  0.015789873128313918
Percent:  0.010506736611634595
Percent:  0.008567372546239583
Percent:  0.006984285232101585
Percent:  0.0056923723052518015
Percent:  0.004638351554051979
Percent:  0.0037786327366897297
Percent:  0.0030775679617718226
Percent:  0.002506014581420047
Percent:  0.0020401556847770637
Percent:  0.0016605326620416205
Perc

<Figure size 640x480 with 1 Axes>